In [7]:
import os
import boto3
import json
import pandas

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth


model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"

index_name =  os.environ.get('AWS_OPENSEARCH_INDEX_NAME')
service = 'aoss'
host = os.getenv('AWS_OPENSEARCH_ENDPOINT')
region = os.getenv('AWS_REGION')


df = pandas.read_parquet('../data/bger-2024-3-text.parquet')


In [4]:
session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWSV4SignerAuth(credentials, region, service)

sts = session.client("sts")
identity = sts.get_caller_identity()
print("Running as IAM identity:", identity["Arn"])

aoss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_compress=True,
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=20,
    timeout=36000,  # Set request timeout in seconds
)

# Check if index exists
try:
    exists = aoss_client.indices.exists(index=index_name)
    print(f"Index {index_name} exists: {exists}")
    if not exists:
        raise {"statusCode": 500, "body": json.dumps(f"Index {index_name} does not exist")}
except Exception as e:
    print(f"Error checking index: {str(e)}")
    raise {"statusCode": 500, "body": json.dumps(f"Error checking index: {str(e)}")}

Running as IAM identity: arn:aws:iam::211125557955:user/Christophe
Index fed-court-chunks-index exists: True


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_name)

test_cases = [
  {
    "title": "Litige transfrontalier en matière de protection des données",
    "content": "Une entreprise suisse basée à Zurich a transféré des données personnelles de ses clients vers un prestataire basé en Floride. Un client suisse s'est plaint que ses données aient été utilisées à des fins de marketing sans consentement explicite. L’affaire soulève la question de la conformité au droit suisse sur la protection des données et au principe du transfert international."
  },
  {
    "title": "Discrimination dans un marché réglementé",
    "content": "Un ingénieur software demandeur d'emploi de nationalité allemande affirme que sa candidature a été rejetée par une entreprise semi-publique suisse en raison de sa nationalité. Il invoque une discrimination contraire à l’accord sur la libre circulation des personnes et aux dispositions suisses sur l’égalité de traitement."
  },
  {
    "title": "Droit des brevets et concurrence déloyale",
    "content": "Une PME suisse accuse une entreprise concurrente d’avoir copié un procédé breveté dans le domaine de la chimie verte, utilisé dans toute la Suisse. Elle saisit le Tribunal fédéral des brevets pour violation de brevet, demandant l’interdiction de vente et des dommages-intérêts."
  },
  {
    "title": "Responsabilité de l’État en matière de cybersécurité",
    "content": "Un citoyen suisse voit ses données fiscales compromises après une cyberattaque sur un portail cantonal de déclaration en ligne. Il intente une action contre la Confédération, affirmant une responsabilité subsidiaire pour défaut de surveillance ou standards insuffisants de cybersécurité imposés aux cantons."
  },
  {
    "title": "Recours contre une décision de l’OFSP",
    "content": "Un thérapeute en médecine complémentaire attaque une décision de l’Office fédéral de la santé publique (OFSP) lui refusant l'autorisation de facturer via l’assurance de base (LAMal). Il estime que la décision viole son droit de pratiquer et demande au Tribunal administratif fédéral d’annuler la décision."
  },
  {
    "title": "Délit d’initié et marchés financiers",
    "content": "Un cadre supérieur d’une banque zurichoise est poursuivi pour avoir vendu des actions la veille d’une annonce de fusion confidentielle. La FINMA a transmis le dossier au Ministère public de la Confédération pour soupçon de délit d’initié, en application de la Loi sur les bourses (LBVM)."
  },
  {
    "title": "Asile et détention administrative",
    "content": "Un ressortissant érythréen, débouté de sa demande d’asile, fait recours contre sa mise en détention en vue du renvoi, décidée par le SEM. Il soutient que son renvoi est impossible et que sa détention viole l’article 5 CEDH et la jurisprudence fédérale sur la proportionnalité."
  },
  {
    "title": "Affaire douanière et taxation à l'importation",
    "content": "Une société suisse importe des pièces électroniques de Chine via un entrepôt sous douane à Bâle. L’Administration fédérale des douanes lui inflige une amende pour fausse déclaration de valeur. L’entreprise fait recours en affirmant avoir suivi les instructions d’un transitaire autorisé."
  },
  {
    "title": "Liberté religieuse et service civil",
    "content": "Un objecteur de conscience témoigne que le refus de son admission au service civil, malgré ses convictions religieuses pacifistes, viole sa liberté de conscience garantie par la Constitution fédérale. L’affaire est portée devant le Tribunal fédéral."
  },
  {
    "title": "Litige en matière d’accès à l’information",
    "content": "Un journaliste demande à l’OFEV l’accès à un rapport interne sur des risques environnementaux liés à une entreprise de recyclage. Le département refuse en invoquant des raisons de confidentialité industrielle. Le journaliste saisit le Préposé fédéral à la transparence puis fait recours devant le Tribunal administratif fédéral."
  }
]


case_number = 3
query_embedding = model.encode([test_cases[case_number]["content"]])[0]

In [8]:
top_matches = 10
response = aoss_client.search(
    index=index_name,
    body={
        "size": top_matches,
        "query": {
            "knn": {
                "embedding": {
                    "vector": query_embedding.tolist(),
                    "k": top_matches
                }
            }
        }
    },
    timeout=30
)
hit_ids = set()
for hit in sorted(response["hits"]["hits"], key=lambda x: x["_source"]["doc_id"], reverse=True):
    hit_ids.add(hit["_source"]["doc_id"])
    print(hit["_source"]["doc_id"], hit["_score"])


4A_406/2014 0.6181955
2F_19/2013 0.6034349
2C_791/2021 0.60196644
2C_653/2018 0.6193916
2C_573/2020 0.6128537
2C_541/2017 0.60512704
2C_337/2021 0.6131343
2C_310/2021 0.60727316
1D_1/2019 0.61250913
1C_381/2009 0.60858536


In [9]:

relevant_docs = [
    {"doc": doc, "ref": doc_ref}
    for doc, doc_ref in zip(
        df.loc[df["docref"].isin(hit_ids), "text"],
        df.loc[df["docref"].isin(hit_ids), "docref"]
    )
]
relevant_docs

[{'doc': 'Bundesgericht Tribunal fédéral Tribunale federale Tribunal federal {T 0/2} 1C_381/2009 Urteil vom 13. Oktober 2009 I. öffentlich-rechtliche Abteilung Besetzung Bundesrichter Féraud, Präsident, Bundesrichter, Aemisegger, Reeb, Fonjallaz, Eusebio, Gerichtsschreiber Härri. Parteien X.________, Beschwerdeführer, vertreten durch Advokat Alain Joset, gegen Bundesamt für Justiz, Fachbereich Auslieferung, Bundesrain 20, 3003 Bern. Gegenstand Auslieferungshaft; Ersatzmassnahmen; "Electronic Monitoring", Beschwerde gegen den Entscheid vom 19. August 2009 des Bundesstrafgerichts, II. Beschwerdekammer. Sachverhalt: A. Am 30. März 2009 ersuchten die italienischen Behörden die Schweiz und andere an das Schengener Informationssystem angeschlossene Staaten um die Inhaftierung des italienischen Staatsangehörigen X.________ (geb. 1973) zwecks Auslieferung; dies gestützt auf den Haftbefehl des Tribunale di Catania vom 2. Februar 2009 wegen Beteiligung an einer kriminellen Organisation und Droge

In [10]:
import openai
import os

api_key = os.environ["OPENAI_API_KEY"]

oai_client = openai.OpenAI(api_key=api_key)
# Set the OpenAI API key
question = f"Fournis un avis juridique en te fondant sur les documents joints concernant le cas suivant : {test_cases[case_number]['content']}"

# Combine the documents into a single prompt
context = "\n\n".join([f"Décision {doc['ref']}:\n{doc['doc']}" for doc in relevant_docs])

response = oai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a legal assistant who answers based only on provided documents."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


Sur la base des décisions fédérales suisses fournies, voici un avis juridique concernant la situation d’un citoyen suisse dont les données fiscales ont été compromises suite à une cyberattaque sur un portail cantonal de déclaration en ligne, et qui engage une action contre la Confédération pour responsabilité subsidiaire liée à un défaut de surveillance ou à des standards insuffisants de cybersécurité imposés aux cantons.

1. **Compétence et responsabilité de la Confédération en matière de protection des données et cybersécurité**

- La décision 4A_406/2014 (protection des données) illustre que la protection des données personnelles est régie par la loi fédérale sur la protection des données (LPD) et que les autorités fédérales et cantonales doivent respecter les principes de proportionnalité, de transparence et de sécurité des données. La Confédération, par ses organes, est tenue de garantir un niveau adéquat de protection des données, notamment dans le cadre de l’échange d’informatio

In [7]:
refs = df["docref"].tolist()
refs

['1B_8/2007',
 '1B_19/2007',
 '1C_16/2007',
 '5A_39/2007',
 '2C_29/2007',
 '1F_2/2007',
 '1B_12/2007',
 '6B_7/2007',
 '5D_4/2007',
 '1B_10/2007',
 '1B_20/2007',
 '5A_41/2007',
 '4A_4/2007',
 '2C_12/2007',
 '2C_22/2007',
 '9C_7/2007',
 '6B_4/2007',
 '1B_6/2007',
 '6B_5/2007',
 '5A_7/2007',
 '6B_1/2007',
 '5A_13/2007',
 '1B_4/2007',
 '2C_7/2007',
 '1B_2/2007',
 '5D_1/2007',
 '1B_5/2007',
 '5F_2/2007',
 '1B_3/2007',
 '5A_21/2007',
 '5A_1/2007',
 '5A_5/2007',
 '2C_4/2007',
 '1F_3/2007',
 '2C_1/2007',
 '1C_1/2007',
 '2F_1/2007',
 '1B_1/2007',
 '6B_76/2007',
 '6B_117/2007',
 '6B_118/2007',
 '6B_106/2007',
 '6B_60/2007',
 '6B_44/2007',
 '9C_140/2007',
 '1C_15/2007',
 '1B_67/2007',
 '5A_44/2007',
 '4A_13/2007',
 '1C_73/2007',
 '9C_130/2007',
 '5A_146/2007',
 '2F_2/2007',
 '5D_16/2007',
 '2C_142/2007',
 '2C_148/2007',
 '6B_61/2007',
 '9C_31/2007',
 '5A_119/2007',
 '1B_44/2007',
 '1B_51/2007',
 '9F_2/2007',
 '4D_7/2007',
 '2C_58/2007',
 '5A_47/2007',
 '1C_66/2007',
 '2C_132/2007',
 '2C_83/2007',